# Task1. 预测是酶还是非酶

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-04-27  

## 任务简介
该任务通过给定蛋白序列，预测该该蛋白是酶还是非酶。本任务所使用的数据集为Sport，对数据集的数据中进行学习，然后对新给定的蛋白序列数据预测是酶还是非酶。


## 数据统计
- 数据源Sprot，共有数据564,638条，其中有EC号的数据270,236条，无EC号的数据294402条。
- 将数据集中的所有数据按照时间排序，～90%作为训练集，～10%作为测试集，找到对应时间节点为2010年2月9日。
- 以2010年2月10日为时间节点，之前的数据为训练集，之后的数据为测试集，具体数据集统计如下： 





|     Items    | 酶     |   非酶    |合计                            |
| ------------ | -------| -------- |-------------------------------|
| 训练集        | 245771 | 264719   | 510490（510490/564638≈90.41%） |
| 测试集        | 24465  | 29683    | 54148（54148/564638≈9.59%）    |

## 数据集构建方法

* 根据蛋白注释信息，将Sprot中的蛋白分为「酶」和 「非酶」。
* 为了保证蛋白的唯一性，按照序列对Sprot中的数据进行去重除了，以保证数据集中序列的唯一性。
* 将注释中含有3级及三级以上EC号的蛋白做为酶数据，收录于数据集中，当中正样本 。
* 将注释中不含有EC号的数据作为非酶数据，并进行如下处理后收录于数据集中，当做是负样本。
> 1. 非酶数据与酶数据进行同源比对(diamoind)，过滤掉相似性>40%的数据
> 2. 过滤掉<50bp的核算片段
> 3. 对酶数据进行CD——HIT比较，按照40%的阈值进行去冗余<span style='background:yellow'>（？有必要吗）</span>

<span style='font-family:"Source Code Pro"; background:yellow'>We adopt three datasets in this paper</span><span  style='font-family:"Source Code Pro"'>. <span style='background: aqua'>The first dataset is a widely used one from (Shen and Chou, 2007), constructed from the ENZYME database (released on May 1, 2007)</span>, <span style='background:silver'>with 40% sequence similarity cutoff</span>. More details of that dataset could be referred to (Shen and Chou, 2007). This dataset is denoted as the KNN dataset in the rest of the paper.</span>

Following the same procedure of constructing the KNN dataset, we constructed a larger dataset using up-to-date databases. The steps of constructing the dataset are as follows:

i. <span style='color:red;background:silver'>The SWISS-PROT (released on September 7, 2016) database was separated into enzymes and non-enzymes based on the
annotation.</span>

ii. <u>To guarantee the uniqueness and correctness</u>, <span style='background:lime'>enzyme sequences with more than one set of EC numbers</span> or <span style='background:red'>incomplete EC number annotation</span> <u>were excluded</u>.</span>

iii. To avoid fragment data, enzyme sequences annotated with ‘fragment’ or <span style='background:red'>with &lt;50 amino acids were excluded</span>. Enzyme sequences with <span style='color:white;background:teal'>more than 5000 amino
acids were also excluded.</span>

iv. <span style='color:white;background:purple'>To remove redundancy bias, we used CD-HIT (Fu et al., 2012) with 40% similarity threshold to sift upon the raw dataset,
resulting in 22168 low-homology enzyme sequences.</span>

v. To construct the <span style='background:yellow'>non-enzyme part</span>, 22168 non-enzyme protein sequences were <span style='background:yellow'>randomly
collected</span> from the SWISS-PROT (released on September 7, 2016) non-enzyme part, <span style='background:fuchsia'>which were also subject to the (ii–iv) steps.</span>

## 实验方法

- 同源比对：使用训练集建立比对库，然后将测试集与之比对，取最优的比对结果，比对结果的（酶/非酶）属性当作测试集的测试结果
- 传统机器学习方法
- 深度学习方法

CUTOFF -< aad 有的是酶，但是没有没有酶号，这个标签需要重新确定（提高数据的质量），不同数据源得到的酶号可能不一样（而且有例子），多数据源联合比对，需要golden standard，

需要Ground truth

## 实验结果

### 1. 未对序列长度进行过滤

|Methods   | Accuracy                        |             Precision           |           Recall               |F1   |Group|
| ---------| ------------------------------- | ------------------------------- |--------------------------------|-----|      |
| 同源比对  |  0.6243628573539189(33808/54148) | 0.8220590380781014(33808/41126) |0.7595109699342543(41126/54148)|      |NO LENGTH FILTERING|
lr 	|	0.619377 |	0.562195 |	0.712160 |	0.628354 | NO LENGTH FILTERING |
xg 	|	0.678363 |	0.647462 |	0.632536 |	0.639912 | NO LENGTH FILTERING |
dt 	|	0.640725 |	0.605286 |	0.588759 |	0.596909 | NO LENGTH FILTERING |
rf 	|	0.696905 |	0.654373 |	0.697650 |	0.675319 | NO LENGTH FILTERING |
gbdt|	0.633024 |	0.571013 |	0.754956 |	0.650226 | NO LENGTH FILTERING |




400 长度


|baslineName| 	 accuracy 	 |precision(PPV)| 	 NPV |		 recall |	 f1 	|	 auroc 	|	 auprc 	|	 confusion Matrix  | 
|-----|-----|-----|-----|-----|-----|-----|-----|-----|
lr 		|0.622017 	|0.565314 		|0.695735 	|0.707214 	|0.628352 	|0.679701 	|0.574374 	 |tp: 17302 fp: 13304 fn: 7163 tn: 16379    
xg 		|0.677532 	|0.647106 		|0.701404 	|0.629675 	|0.638271 	|0.755010 	|0.687809 	 |tp: 15405 fp: 8401 fn: 9060 tn: 21282    
dt 		|0.639174 	|0.603897 		|0.666656 	|0.585285 	|0.594445 	|0.634438 	|0.540833 	 |tp: 14319 fp: 9392 fn: 10146 tn: 20291   
rf 		|0.696997 	|0.654997 		|0.735775 	|0.695933 	|0.674845 	|0.790091 	|0.739525 	 |tp: 17026 fp: 8968 fn: 7439 tn: 20715  


700 长度

|baslineName| 	 accuracy 	 |precision(PPV)| 	 NPV |		 recall |	 f1 	|	 auroc 	|	 auprc 	|	 confusion Matrix  | 
|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|lr 		|0.622017 	|0.565314 		|0.695735 	|0.707214 	|0.628352 	|0.679701 	|0.574374 	 |tp: 17302 fp: 13304 fn: 7163 tn: 16379  
|xg 		|0.677532 	|0.647106 		|0.701404 	|0.629675 	|0.638271 	|0.755010 	|0.687809 	 |tp: 15405 fp: 8401 fn: 9060 tn: 21282   
|dt 		|0.639174 	|0.603897 		|0.666656 	|0.585285 	|0.594445 	|0.634438 	|0.540833 	 |tp: 14319 fp: 9392 fn: 10146 tn: 20291   
|rf 		|0.696997 	|0.654997 		|0.735775 	|0.695933 	|0.674845 	|0.790091 	|0.739525 	 |tp: 17026 fp: 8968 fn: 7439 tn: 20715   




### 2. 对序列长度进行过滤【60，1000】

> <60 占比 2.9878%   
> \>1000 占比 3.2761%

|Methods   | Accuracy                        |             Precision           |           Recall               |F1   |Group|
| ---------| ------------------------------- | ------------------------------- |--------------------------------|-----|      |
| 同源比对  |  0.6290837227494493(30559/48577) | 0.8195837579788661(30559/37286) |0.7675648969677008(37286/48577)|      |60-1000|
|lr 				| 0.614159 	|0.565746 		 	|0.746395 	|0.643635 |60-1000|
|xg 				| 0.672170 	|0.649420 		 	|0.647043 	|0.648230 |60-1000|
|dt 				|0.625996 	|0.601167 		 	|0.590775 	|0.595925 |60-1000|
|rf 				|0.691253 	|0.640584 		 	|0.771487 	|0.699968 |60-1000|
|gbdt 			|0.627869 	|0.573841 		 	|0.788199 	|0.664152 |60-1000|

Total query records are: 48577
Matched records are: 37286
Accuracy: 0.6290837227494493(30559/48577)
Pricision: 0.8195837579788661(30559/37286)
Recall: 0.7675648969677008(37286/48577)

## 1. 导入必要的包、定义公共函数

In [ ]:
import numpy as np
import pandas as pd
import random
import time
import gzip
import re
from Bio import SeqIO
import datetime
import sys
from tqdm import tqdm

from functools import reduce
import matplotlib.pyplot as plt

sys.path.append("../../tools/")
import commontools
import funclib
from pyecharts.globals import CurrentConfig, OnlineHostType
CurrentConfig.ONLINE_HOST = OnlineHostType.NOTEBOOK_HOST
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.faker import Faker
from pyecharts.globals import ThemeType

from jax_unirep import get_reps

# Thres 》=100 《=600｜500

## 2. 加载数据

In [35]:
table_head = [  'id', 
                'isemzyme',
                'isMultiFunctional', 
                'functionCounts', 
                'ec_number', 
                'date_integraged',
                'date_sequence_update',
                'date_annotation_update',
                'seq', 
                'seqlength'
            ]

#加载数据并转换时间格式
sprot = pd.read_csv('../../data/sprot_full.tsv', sep='\t',names=table_head) #读入文件
sprot.date_integraged = pd.to_datetime(sprot['date_integraged'])
sprot.date_sequence_update = pd.to_datetime(sprot['date_sequence_update'])
sprot.date_annotation_update = pd.to_datetime(sprot['date_annotation_update'])

sprot.head(2)

,id,isemzyme,isMultiFunctional,functionCounts,ec_number,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,P02802,False,False,1,-,1986-07-21,1986-07-21,2021-04-07,MDPNCSCSTGGSCTCTSSCACKNCKCTSCKKSCCSCCPVGCSKCAQ...,61
1,P02732,False,False,1,-,1986-07-21,1986-07-21,2019-12-11,AATAATAATAATAATAATAATAATAATAATA,31


## 3. 统计数据
### 3.1 序列长度统计

In [5]:
sprot_length = sprot.seqlength.value_counts(ascending=True)
sprot_length = pd.DataFrame(sprot_length)
sprot_length['length'] = sprot_length.index
sprot_length = sprot_length.rename(columns={'seqlength':'count', 'length':'length'})
sprot_length=sprot_length.sort_values(by='length', ascending=True)
sprot_length=sprot_length.loc[:,['length','count']]
sprot_length.head(3)

,length,count
2,2,2
3,3,5
4,4,22


In [1]:
span = 20
res_statistic = pd.DataFrame(funclib.static_interval(sprot_length, span))

bar = (
    Bar(init_opts=opts.InitOpts(width="1700px",
                                height="750px",
                                page_title="造价四剑客",
                                theme=ThemeType.CHALK))
    .add_xaxis(list(res_statistic[0]))
    .add_yaxis("序列长度", list(res_statistic[1]))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="SPROT 数据集中蛋白序列长度区间统计图，统计区间跨度=20"),
        datazoom_opts=opts.DataZoomOpts()
    )
)

### 2  加载javascript
bar.load_javascript()
bar.render_notebook()

NameError: name 'pd' is not defined

In [37]:
l_thres = 60
h_thres = 1000
total = len(sprot)

l_num = sprot_length[sprot_length.length < l_thres]['count'].sum()
h_num = sprot_length[sprot_length.length > h_thres]['count'].sum()


print('<{} 占比 {:5.4f}%'.format(l_thres, l_num/total *100))
print('>{} 占比 {:5.4f}%'.format(h_thres, h_num/total *100))

<60 占比 2.9878%
>1000 占比 3.2761%


## 3. 划分训练集、测试集

In [31]:
sprot_exp = sprot[(sprot.seqlength >= l_thres) & (sprot.seqlength <= h_thres) ]

t_thres = datetime.datetime(2010, 2, 10, 0, 0)

#训练集
train = sprot_exp[sprot_exp.date_integraged <= t_thres ].sort_values(by='date_integraged')
#测试集
test = sprot_exp[sprot_exp.date_integraged > t_thres ].sort_values(by='date_integraged')

# train.to_csv('./data/train.tsv', sep='\t', columns=['id', 'isemzyme','seq'], index=0)
# test.to_csv('./data/test.tsv', sep='\t', columns=['id', 'isemzyme','seq'], index=0)

# funclib.table2fasta(train, './data/train.fasta')
# funclib.table2fasta(test, './data/test.fasta')

## 4. 二分类
### 4.1 同源比对

In [ ]:
! diamond makedb --in ./data/train.fasta -d ./data/train.dmnd     #建库
! diamond blastp -d ./data/train.dmnd  -q ./data/test.fasta -o ./data/test_fasta_results.tsv -b5 -c1 -k 1   #生成比对文件

In [76]:
#读入比对结果
res_data = pd.read_csv('./data/test_fasta_results.tsv', sep='\t', names=['id', 'sseqid', 'pident', 'length','mismatch','gapopen','qstart','qend','sstart','send','evalue','bitscore'])

#匹配查询结果
data_match = pd.merge(test,res_data, on=['id'], how='inner')

In [77]:
# 添加查询结果的EC号
counter =0
resArray =[]
for i in range(len(res_data)):
    counter+=1
    mn = train[train['id']== res_data['sseqid'][i]]['ec_number'].values
    resArray.append(mn)
    if counter %1000 ==0:
        print(counter)
data_match['sresults_ec']=np.array(resArray) 
data_match.head(3)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000


,id,isemzyme,isMultiFunctional,functionCounts,ec_number,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength,...,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,sresults_ec
0,B3M1H7,False,False,1,-,2010-03-02,2008-09-02,2020-12-02,MESLSQLVKSTLPNYLSNLPIPDSVGGWFKLSFKDWLALIPPTVVV...,134,...,133,23,0,1,133,1,133,3.750000e-88,254.0,-
1,A8Q8J2,False,False,1,-,2010-03-02,2008-01-15,2021-04-07,MELSESTKKALKAIPLMRTKAGPRDGDSWVQRLKEEYEALIAFINN...,165,...,159,45,0,5,163,4,162,1.360000e-92,268.0,-
2,C7GQV5,True,False,1,3.6.1.29,2010-03-02,2010-03-02,2021-04-07,MNKPIYFSKFLVTEQVFYKSKYTYALVNLKPIVPGHVLIVPLRTTV...,206,...,206,0,0,1,206,1,206,1.640000e-149,416.0,3.6.1.29


In [78]:
# 计算指标
data_match['iscorrect'] = data_match[['ec_number', 'sresults_ec']].apply(lambda x: x['ec_number'] == x['sresults_ec'], axis=1) #判断EC号是否一致
correct = sum(data_match['iscorrect'])
find  = len(data_match)
total = len(test)
print('Total query records are: {0}'.format(total))
print('Matched records are: {0}'.format(find))
print('Accuracy: {0}({1}/{2})'.format(correct/total, correct, total))
print('Pricision: {0}({1}/{2})'.format(correct/find, correct, find))
print('Recall: {0}({1}/{2})'.format(find/total, find, total))

Total query records are: 48577
Matched records are: 37286
Accuracy: 0.6290837227494493(30559/48577)
Pricision: 0.8195837579788661(30559/37286)
Recall: 0.7675648969677008(37286/48577)


In [80]:
sprot_exp.seqlength.describe()

count    529270.000000
mean        328.990479
std         195.433691
min          60.000000
25%         177.000000
50%         293.000000
75%         435.000000
max        1000.000000
Name: seqlength, dtype: float64

### 4.2 使用机器学习方法

In [9]:
trainset = train[['id', 'isemzyme','seq', 'seqlength']].reset_index(drop=True)
testset = test[['id', 'isemzyme','seq', 'seqlength']].reset_index(drop=True)

# MAX_SEQ_LENGTH = 1000 #定义序列最长的长度
# trainset.seq = trainset.seq.map(lambda x : x[0:MAX_SEQ_LENGTH].ljust(MAX_SEQ_LENGTH, 'X'))
# testset.seq = testset.seq.map(lambda x : x[0:MAX_SEQ_LENGTH].ljust(MAX_SEQ_LENGTH, 'X'))

In [10]:
f_train = funclib.dna_onehot(trainset) #训练集编码
f_test = funclib.dna_onehot(testset) #测试集编码

train_full = pd.concat([trainset, f_train], axis=1, join='inner' ) #拼合训练集
test_full = pd.concat([testset, f_test], axis=1, join='inner' )    #拼合测试集

In [11]:
X_train = train_full.iloc[:,4:]
X_test = test_full.iloc[:,4:]
Y_train = train_full.isemzyme.astype('int')
Y_test = test_full.isemzyme.astype('int')

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [ ]:
methods=['lr', 'xg', 'dt', 'rf', 'gbdt']
print('baslineName', '\t', 'accuracy','\t', 'precision(PPV) \t NPV \t\t', 'recall','\t', 'f1', '\t\t', 'auroc','\t\t', 'auprc', '\t\t confusion Matrix')
for method in methods:
    funclib.evaluate(method, X_train, Y_train, X_test, Y_test)

baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix
lr 		0.614694 	0.568831 		0.681288 	0.721568 	0.636160 	0.666294 	0.579905 	 tp: 16363 fp: 12403 fn: 6314 tn: 13497
[16:07:56] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
xg 		0.670605 	0.648659 		0.689469 	0.642281 	0.645454 	0.747822 	0.693614 	 tp: 14565 fp: 7889 fn: 8112 tn: 18011
dt 		0.629495 	0.605616 		0.649512 	0.591569 	0.598510 	0.627112 	0.548915 	 tp: 13415 fp: 8736 fn: 9262 tn: 17164
rf 		0.694526 	0.651876 		0.742853 	0.741765 	0.693921 	0.782771 	0.743850 	 tp: 16821 fp: 8983 fn: 5856 tn: 16917


## 5. Unirep 特征

In [10]:
t_thres = datetime.datetime(2010, 2, 10, 0, 0)

#训练集
train = sprot[sprot.date_integraged <= t_thres ].sort_values(by='date_integraged')
#测试集
test = sprot[sprot.date_integraged > t_thres ].sort_values(by='date_integraged')

In [10]:
X_train = trainset['seq']
Y_train = trainset.isemzyme.astype('int')

X_test = testset['seq']
Y_test = testset.isemzyme.astype('int')

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [14]:
train['label'] = train.isemzyme.astype('int')
test['label'] = test.isemzyme.astype('int')

In [18]:
trainset = train[['seq', 'label']]
testset = test[['seq', 'label']]

In [23]:
step = 2000
!rm -rf ./data/sprot/train_*.tsv
counter = 1
for i in range(0, len(trainset), step):
    save = trainset[i:i+step].iloc[:,0:2]
    save.to_csv('./data/sprot/train_{0}.tsv'.format(counter), sep='\t',  index=0)
    counter +=1

In [22]:
step = 2000
!rm -rf ./data/sprot/test_*.tsv
counter = 1
for i in range(0, len(testset), step):
    save = testset[i:i+step].iloc[:,0:2]
    save.to_csv('./data/sprot/test_{0}.tsv'.format(counter), sep='\t',  index=0)
    counter +=1